In [11]:
import cv2
import numpy as np
import random

In [12]:
raw_cat = cv2.imread("./rawdata/cat.png", 0)
raw_dog = cv2.imread("./rawdata/dog.png", 0)

### ERROR DIFFUSION

In [13]:
def error_dissusion(img):
    mask = 1/48 * np.array([[0,0,0,7,5],[3,5,7,5,3],[1,3,5,3,1]])
    row_num, col_num = img.shape        
    w = (mask.shape[1]-1)//2
    result = img.copy().astype(float)
    for i in range(0, row_num-w):
        for j in range(w, col_num-w):
            value = result[i, j]
            result[i, j] = 255 if value > 127.5 else 0
            error = value - result[i, j]
            result[i:i+w+1, j-w:j+w+1] += error*mask
    return result

In [14]:
cat_ED = error_dissusion(raw_cat)
dog_ED = error_dissusion(raw_dog)
cv2.imwrite("./output/cat_ED.png", cat_ED)
cv2.imwrite("./output/dog_ED.png", dog_ED)

True

### VISUAL CRYPOTOGRAPHY

In [15]:
# 1是黑 0是白
pattern1 = np.array([[0,0],[1,1]])
pattern2 = np.array([[1,1],[0,0]])
pattern3 = np.array([[0,1],[0,1]])
pattern4 = np.array([[1,0],[1,0]])
pattern5 = np.array([[0,1],[1,0]])
pattern6 = np.array([[1,0],[0,1]])
patterns = [pattern1, pattern2, pattern3, pattern4, pattern5, pattern6]

In [16]:
def visual_cryptography2x2(img, patterns=patterns):
    share1 = np.zeros((img.shape[0]*2, img.shape[1]*2))
    share2 = np.zeros((img.shape[0]*2, img.shape[1]*2))
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            x = random.randint(0, 5)
            if img[i,j] == 0: #若原始pixel為黑色
                if(x % 2 == 1): # 1, 3, 5
                    share1[i*2:i*2+2, j*2:j*2+2] = patterns[x]
                    share2[i*2:i*2+2, j*2:j*2+2] = patterns[x-1]
                else: # 0, 2, 4
                    share1[i*2:i*2+2, j*2:j*2+2] = patterns[x]
                    share2[i*2:i*2+2, j*2:j*2+2] = patterns[x+1]           
            elif img[i,j] == 255: #若原始pixel為白色
                share1[i*2:i*2+2, j*2:j*2+2] = patterns[x]
                share2[i*2:i*2+2, j*2:j*2+2] = patterns[x]
    result = np.logical_or(share1.astype(bool), share2.astype(bool)).astype(int)
    result[result==0]=255
    result[result==1]=0
    share1[share1==0]=255
    share1[share1==1]=0
    share2[share2==0]=255
    share2[share2==1]=0
    return share1, share2, result

In [17]:
share1, share2, cat_VC = visual_cryptography2x2(cat_ED)
cv2.imwrite("./output/cat_VC.png", cat_VC)
cv2.imwrite("./output/share1.png", share1)
cv2.imwrite("./output/share2.png", share2)

True

### Inverse

In [18]:
def pattern_type(matrix, patterns=patterns):
    for i in range(len(patterns)):
        if (patterns[i]==matrix).all():
            return i    
    return random.randint(0, 5)

In [21]:
def Inverse(img, key, patterns=patterns):
    share3 = np.zeros((key.shape[0], key.shape[1]))
    key = key.astype(int)
    key[key==0]=1
    key[key==255]=0
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            x = pattern_type(key[i*2:i*2+2, j*2:j*2+2])
            if img[i,j] == 0: #若原始pixel為黑色
                if(x % 2 == 1): # 1, 3, 5
                    share3[i*2:i*2+2, j*2:j*2+2] = patterns[x-1]
                else: # 0, 2, 4
                    share3[i*2:i*2+2, j*2:j*2+2] = patterns[x+1]           
            elif img[i,j] == 255: #若原始pixel為白色
                share3[i*2:i*2+2, j*2:j*2+2] = patterns[x]
    result = np.logical_or(key.astype(bool), share3.astype(bool)).astype(int)
    result[result==0]=255
    result[result==1]=0    
    share3[share2==0]=255
    share3[share2==1]=0
#     share3[:,:4] = 255
#     share3[:,508:] = 255
#     share3[508:,:] = 255
    return share3, result

In [22]:
share3, dog_VC = Inverse(dog_ED, share2)
cv2.imwrite("./output/share3.png", share3)
cv2.imwrite("./output/dog_VC.png", dog_VC)

True